In [141]:
import numpy as np
import matplotlib.pylab as plt
from scipy.ndimage.interpolation import zoom as scipy_zoom
from scipy import ndimage, misc
from skimage.color import rgb2hsv, hsv2rgb
from mapreduce import *
from multiprocessing.pool import ThreadPool
from multiprocessing import Pool
from PIL import Image
import time


class AugmentateImage:
    def __init__(self, in_path, out_dir):
        self.im = plt.imread(in_path)
        self.out_dir = out_dir
        self.save_image(self.im, self.out_dir + "/img_init.jpg")
        self.pad()
        
    def run(self):
        if (self.im.shape[0] == 0) or (self.im.shape[0] == 1):
            return
        self.save_image(self.im, self.out_dir + "/img_padded.jpg")
        imcropped = self.crop()
        self.save_image(imcropped, self.out_dir + "/img_cropped.jpg")
        
        def mapper_2(params):
            (zoom1, zoom2, chcolors, fliptype, shiftcolor, skewtype, kskew) = params
            ret = self.zoom(zooms=[zoom1, zoom2], im=None)
            ret = self.change_colors(change=chcolors, im=ret)
            ret = self.flip(fliptype=fliptype, im=ret)
            ret = self.shift_h_color(shift=shiftcolor, im=ret)
            ret = self.skew(skewtype=skewtype, k=kskew, im=ret)
            ret = self.crop(size=256, im=ret)
            
            if not np.array_equal(ret, imcropped):
                self.save_image(ret, self.out_dir + "/img_%.2f_%.2f_%d_%s_%.2f_%s_%.2f.jpg" % (zoom1, zoom2, chcolors[0]*4+chcolors[1]*2+chcolors[2], fliptype, shiftcolor, skewtype, kskew))

        def reducer_2(accumulated, item):
            return None

        mr = MapReducer().mapper(mapper_2).reducer(reducer_2, [])
        allparams = []
        for zoom1 in [0.8, 0.9, 1.]:
            for zoom2 in [0.8, 0.9, 1.]:
                for chcolors in [[2,1,0], [0,2,1], [0,1,2]]:
                    for fliptype in ["up", "lr", "no"]:
                        for shiftcolor in [0., 0.33, 0.66]:
                            for skewtype in ["lr", "tb"]:
                                for kskew in [0., 0.3, 0.6]:
                                    allparams.append((zoom1, zoom2, chcolors, fliptype, shiftcolor, skewtype, kskew))
        result = mr(allparams)
        
    
    def pad(self):
        side = max(self.im.shape[0], self.im.shape[1])
        ret = np.zeros((side, side, 3))
        
        if self.im.shape[0] <= side:
            ret[int((side - self.im.shape[0]) / 2.):int((side - self.im.shape[0]) / 2.) + self.im.shape[0], :, :] = self.im
        else:
            ret[:, int((side - self.im.shape[1]) / 2.):int((side - self.im.shape[1]) / 2.) + self.im.shape[1], :] = self.im
            
        self.im = ret.astype(np.uint8)
        
    def plti(self, h=6, im=None):
        """
        Helper function to plot an image.
        """
        if im is None:
            im = self.im
        y = im.shape[0]
        x = im.shape[1]
        w = (y/x) * h
        plt.figure(figsize=(w,h))
        plt.imshow(im, interpolation='bicubic')
        plt.axis('off')

    def crop(self, size=256, im=None):
        if im is None:
            im = self.im
        first = ":" if im.shape[0] <= size else "int((im.shape[0] - size) / 2):int((im.shape[0] - size) / 2) + size"
        second = ":" if im.shape[1] <= size else "int((im.shape[1] - size) / 2):int((im.shape[1] - size) / 2) + size"
        ret = eval("im[%s, %s, :]" % (first, second))
        return ret
    
    def save_image(self, im, outfilename) :
        im = Image.fromarray(im)
        im.save(outfilename)

    def to_grayscale(self, weights=np.c_[0.33, 0.33, 0.33], im=None):
        """
        Transforms a colour image to a greyscale image by
        taking the mean of the RGB values, weighted
        by the matrix weights
        """
        if im is None:
            im = self.im
        tile = np.tile(weights, reps=(im.shape[0], im.shape[1], 1))
        ret = np.sum(tile * im, axis=2).astype(np.uint8)
        ret = np.repeat(ret, 3). reshape(im.shape)
        return ret

    def zoom(self, zooms=[0.5, 0.5], im=None):
        if im is None:
            im = self.im
        return scipy_zoom(im, (zooms[0], zooms[1], 1))
    
    def change_colors(self, change=[2,1,0], im=None):
        if im is None:
            im = self.im
        im_bgr = im[:, :, change]
        return im_bgr

    def rotate(self, angle=0, im=None):
        if im is None:
            im = self.im
        return ndimage.rotate(im, angle, reshape=False)
    
    def flip(self, fliptype="up", im=None):
        if im is None:
            im = self.im
        if fliptype == "up":
            return np.flipud(im)
        if fliptype == "lr":
            return np.fliplr(im)
        return im

    def noise(self, level=50, im=None):
        if im is None:
            im = self.im
        ret = im.astype(np.int16) + np.random.randint(level, size=im.shape)
        return np.clip(ret, 0, 255).astype(np.uint8)
    
    def shift_h_color(self, shift=0., im=None):
        if im is None:
            im = self.im
        im_hsv = rgb2hsv(im)
        im_hsv[:,:,0] += shift
        return (hsv2rgb(im_hsv)*255).astype(np.uint8)

    def skew(self, skewtype="lr", k=0.1, im=None):
        if im is None:
            im = self.im

        ih = int(im.shape[0] / 2.)
        jh = int(im.shape[1] / 2.)
        ret = np.zeros(im.shape)
        for i in range(im.shape[0]):
            for j in range(im.shape[1]):
                i_src = i
                j_src = j
                if skewtype == "lr":
                    j_src = int(j + (i - ih) * k)
                if skewtype == "tb":
                    i_src = int(i - (j - jh) * k)
                if (i_src < 0) or (i_src >= im.shape[0]) or (j_src < 0) or (j_src >= im.shape[1]):
                    ret[i, j, :] = [0, 0, 0]
                else:
                    ret[i, j, :] = im[i_src, j_src, :]

        return ret.astype(np.uint8)


    

In [142]:
augi = augmentate_image("./img/img_1.jpg", "./img_aug")
augi.run()

Process Process-187:
Process Process-172:
Traceback (most recent call last):
Process Process-193:
Process Process-194:
Traceback (most recent call last):
Process Process-184:
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process Process-174:
Process Process-169:
Process Process-189:
Process Process-186:
Process Process-185:
Process Process-197:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lo

KeyboardInterrupt: 

  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process Process-179:
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/math/.local/lib/python3.7/site-packages/mapreduce/mapreduce.py", line 136, in _process
    new_item = self._

  File "/home/math/.local/lib/python3.7/site-packages/mapreduce/mapreduce.py", line 136, in _process
    new_item = self._mapper(item)
KeyboardInterrupt
  File "/home/math/.local/lib/python3.7/site-packages/mapreduce/mapreduce.py", line 136, in _process
    new_item = self._mapper(item)
KeyboardInterrupt
  File "<ipython-input-141-151eb0eb309b>", line 33, in mapper_2
    ret = self.skew(skewtype=skewtype, k=kskew, im=ret)
  File "<ipython-input-141-151eb0eb309b>", line 154, in skew
    j_src = int(j + (i - ih) * k)
  File "<ipython-input-141-151eb0eb309b>", line 154, in skew
    j_src = int(j + (i - ih) * k)
  File "<ipython-input-141-151eb0eb309b>", line 33, in mapper_2
    ret = self.skew(skewtype=skewtype, k=kskew, im=ret)
KeyboardInterrupt
KeyboardInterrupt
  File "<ipython-input-141-151eb0eb309b>", line 156, in skew
    i_src = int(i - (j - jh) * k)
  File "<ipython-input-141-151eb0eb309b>", line 33, in mapper_2
    ret = self.skew(skewtype=skewtype, k=kskew, im=ret)
  File "<ipyt

  File "<ipython-input-141-151eb0eb309b>", line 107, in zoom
    return scipy_zoom(im, (zooms[0], zooms[1], 1))
  File "/home/math/.local/lib/python3.7/site-packages/scipy/ndimage/interpolation.py", line 623, in zoom
    _nd_image.zoom_shift(filtered, zoom, None, output, order, mode, cval)
KeyboardInterrupt
  File "<ipython-input-141-151eb0eb309b>", line 107, in zoom
    return scipy_zoom(im, (zooms[0], zooms[1], 1))
  File "/home/math/.local/lib/python3.7/site-packages/scipy/ndimage/interpolation.py", line 623, in zoom
    _nd_image.zoom_shift(filtered, zoom, None, output, order, mode, cval)
KeyboardInterrupt
Process Process-195:
Process Process-181:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 9